In [2]:
import pandas as pd
import json
import math

In [3]:
df = pd.read_json("./experiment_queries_execution_no_rerank2.json", orient="records")
df
    

,method_id,user_id,class_id,query_id,class_category,rr,time
0,single_elastic_bm25,1,20825628,user_query_0,instances_100_1000,0.00,0.115999
1,single_elastic_bm25,1,20825628,user_query_1,instances_100_1000,0.00,0.069224
2,single_elastic_bm25,1,20825628,user_query_2,instances_100_1000,0.00,0.050538
3,single_elastic_bm25,1,20825628,user_query_3,instances_100_1000,0.00,0.081411
4,single_elastic_bm25,1,20825628,user_query_4,instances_100_1000,0.00,0.255341
...,...,...,...,...,...,...,...
99523,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,114,23442,mech_label_user_query_1,instances_100000_1000000000,1.00,0.088526
99524,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,114,23442,mech_label_user_query_2,instances_100000_1000000000,1.00,0.079080
99525,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,114,23442,mech_label_user_query_3,instances_100000_1000000000,0.50,0.084768
99526,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,114,23442,mech_label_user_query_4,instances_100000_1000000000,0.25,0.089515


# Overall

In [4]:
df_thin = df[["method_id", "rr", "time"]]
df_thin.head()

,method_id,rr,time
0,single_elastic_bm25,0.0,0.115999
1,single_elastic_bm25,0.0,0.069224
2,single_elastic_bm25,0.0,0.050538
3,single_elastic_bm25,0.0,0.081411
4,single_elastic_bm25,0.0,0.255341


In [5]:
df_overall =  df_thin.groupby(["method_id"], as_index=False).mean().sort_values(by="rr", ascending=False)
print(df_overall.to_string())

                                                                method_id        rr      time
28  triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.5_x_elastic_bm25_0.1  0.564467  0.179312
27  triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.4_x_elastic_bm25_0.2  0.561279  0.180053
23  triple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.6_x_elastic_bm25_0.1  0.560971  0.177657
22  triple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.5_x_elastic_bm25_0.2  0.560089  0.180253
32  triple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.4_x_elastic_bm25_0.1  0.558781  0.180205
46                      tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6  0.557655  0.180557
16  triple_fusion_qdrant_dense_0.2_x_qdrant_sparse_0.6_x_elastic_bm25_0.2  0.555713  0.178954
17  triple_fusion_qdrant_dense_0.2_x_qdrant_sparse_0.7_x_elastic_bm25_0.1  0.555577  0.178116
31  triple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.3_x_elastic_bm25_0.2  0.554010  0.182489
48                      tuple_fusion_qdrant_dense_0.5_x_qdra

### Overall only tuples

In [6]:
df_overall_tuple_only = df_overall[df_overall["method_id"].str.contains("tuple")]
df_overall_tuple_only

,method_id,rr,time
46,tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6,0.557655,0.180557
48,tuple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.5,0.553837,0.175547
44,tuple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.7,0.553140,0.177974
50,tuple_fusion_qdrant_dense_0.6_x_qdrant_sparse_0.4,0.549166,0.178436
42,tuple_fusion_qdrant_dense_0.2_x_qdrant_sparse_0.8,0.547480,0.177617
40,tuple_fusion_qdrant_dense_0.1_x_qdrant_sparse_0.9,0.540570,0.180089
64,tuple_fusion_qdrant_sparse_0.8_x_elastic_bm25_0.2,0.539204,0.108232
65,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,0.538027,0.103789
52,tuple_fusion_qdrant_dense_0.7_x_qdrant_sparse_0.3,0.533588,0.178387
63,tuple_fusion_qdrant_sparse_0.7_x_elastic_bm25_0.3,0.527437,0.107466


### Overall only triples

In [7]:
df_overall_triple_only = df_overall[df_overall["method_id"].str.contains("triple")]
print(df_overall_triple_only.to_string())

                                                                method_id        rr      time
28  triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.5_x_elastic_bm25_0.1  0.564467  0.179312
27  triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.4_x_elastic_bm25_0.2  0.561279  0.180053
23  triple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.6_x_elastic_bm25_0.1  0.560971  0.177657
22  triple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.5_x_elastic_bm25_0.2  0.560089  0.180253
32  triple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.4_x_elastic_bm25_0.1  0.558781  0.180205
16  triple_fusion_qdrant_dense_0.2_x_qdrant_sparse_0.6_x_elastic_bm25_0.2  0.555713  0.178954
17  triple_fusion_qdrant_dense_0.2_x_qdrant_sparse_0.7_x_elastic_bm25_0.1  0.555577  0.178116
31  triple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.3_x_elastic_bm25_0.2  0.554010  0.182489
35  triple_fusion_qdrant_dense_0.6_x_qdrant_sparse_0.3_x_elastic_bm25_0.1  0.549208  0.180948
9   triple_fusion_qdrant_dense_0.1_x_qdrant_sparse_0.7_x_ela

# Overall per category



In [8]:
df_thin_per_category = df[["method_id", "class_category", "rr", "time"]]
df_thin_per_category

,method_id,class_category,rr,time
0,single_elastic_bm25,instances_100_1000,0.00,0.115999
1,single_elastic_bm25,instances_100_1000,0.00,0.069224
2,single_elastic_bm25,instances_100_1000,0.00,0.050538
3,single_elastic_bm25,instances_100_1000,0.00,0.081411
4,single_elastic_bm25,instances_100_1000,0.00,0.255341
...,...,...,...,...
99523,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,instances_100000_1000000000,1.00,0.088526
99524,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,instances_100000_1000000000,1.00,0.079080
99525,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,instances_100000_1000000000,0.50,0.084768
99526,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,instances_100000_1000000000,0.25,0.089515


In [9]:
df_overall_per_category = df_thin_per_category.groupby(["method_id", "class_category"], as_index=False).mean().sort_values(by=["class_category","rr"], ascending=False)
print(df_overall_per_category.to_string())

                                                                 method_id               class_category        rr      time
143  triple_fusion_qdrant_dense_0.2_x_qdrant_sparse_0.7_x_elastic_bm25_0.1           instances_100_1000  0.572621  0.181608
191  triple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.6_x_elastic_bm25_0.1           instances_100_1000  0.572095  0.180832
231  triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.5_x_elastic_bm25_0.1           instances_100_1000  0.567319  0.182463
183  triple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.5_x_elastic_bm25_0.2           instances_100_1000  0.566710  0.184023
135  triple_fusion_qdrant_dense_0.2_x_qdrant_sparse_0.6_x_elastic_bm25_0.2           instances_100_1000  0.565390  0.181989
375                      tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6           instances_100_1000  0.563104  0.183400
263  triple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.4_x_elastic_bm25_0.1           instances_100_1000  0.562916  0.183844
87   tri

### Only instance categories

In [10]:
print(df_overall_per_category[df_overall_per_category["class_category"].str.startswith("instances")].to_string())

                                                                 method_id               class_category        rr      time
143  triple_fusion_qdrant_dense_0.2_x_qdrant_sparse_0.7_x_elastic_bm25_0.1           instances_100_1000  0.572621  0.181608
191  triple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.6_x_elastic_bm25_0.1           instances_100_1000  0.572095  0.180832
231  triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.5_x_elastic_bm25_0.1           instances_100_1000  0.567319  0.182463
183  triple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.5_x_elastic_bm25_0.2           instances_100_1000  0.566710  0.184023
135  triple_fusion_qdrant_dense_0.2_x_qdrant_sparse_0.6_x_elastic_bm25_0.2           instances_100_1000  0.565390  0.181989
375                      tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6           instances_100_1000  0.563104  0.183400
263  triple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.4_x_elastic_bm25_0.1           instances_100_1000  0.562916  0.183844
87   tri

### Only ancestors cetegories

In [11]:
print(df_overall_per_category[df_overall_per_category["class_category"].str.startswith("ancestors")].to_string())

                                                                 method_id            class_category        rr      time
9                                                      single_qdrant_dense  ancestors_180_1000000000  0.812500  0.188149
449                      tuple_fusion_qdrant_dense_0.9_x_qdrant_sparse_0.1  ancestors_180_1000000000  0.812500  0.187990
433                      tuple_fusion_qdrant_dense_0.8_x_qdrant_sparse_0.2  ancestors_180_1000000000  0.809295  0.189488
305  triple_fusion_qdrant_dense_0.8_x_qdrant_sparse_0.1_x_elastic_bm25_0.1  ancestors_180_1000000000  0.802885  0.187152
425                       tuple_fusion_qdrant_dense_0.8_x_elastic_bm25_0.2  ancestors_180_1000000000  0.802885  0.194685
441                       tuple_fusion_qdrant_dense_0.9_x_elastic_bm25_0.1  ancestors_180_1000000000  0.802885  0.187971
417                      tuple_fusion_qdrant_dense_0.7_x_qdrant_sparse_0.3  ancestors_180_1000000000  0.794872  0.190102
289  triple_fusion_qdrant_dense_

### Only children

In [12]:
print(df_overall_per_category[df_overall_per_category["class_category"].str.startswith("children")].to_string())

                                                                 method_id              class_category        rr      time
179  triple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.5_x_elastic_bm25_0.2       children_10000_100000  0.570499  0.181896
219  triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.4_x_elastic_bm25_0.2       children_10000_100000  0.566753  0.182131
131  triple_fusion_qdrant_dense_0.2_x_qdrant_sparse_0.6_x_elastic_bm25_0.2       children_10000_100000  0.564696  0.181742
211  triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.3_x_elastic_bm25_0.3       children_10000_100000  0.558944  0.180831
227  triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.5_x_elastic_bm25_0.1       children_10000_100000  0.558459  0.182583
291  triple_fusion_qdrant_dense_0.7_x_qdrant_sparse_0.1_x_elastic_bm25_0.2       children_10000_100000  0.558155  0.180793
371                      tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6       children_10000_100000  0.556276  0.181473
299  triple_fusi

# Overall - only mechanic queries

In [13]:
df_only_mech_queries = df[df["query_id"].str.startswith("mech")]
df_only_mech_queries

,method_id,user_id,class_id,query_id,class_category,rr,time
672,single_elastic_bm25,0,20825628,mech_label,instances_100_1000,1.00,0.044602
673,single_elastic_bm25,0,20825628,mech_desc,instances_100_1000,0.00,0.126199
674,single_elastic_bm25,0,20825628,mech_label_desc,instances_100_1000,0.50,0.139791
675,single_elastic_bm25,101,20825628,mech_label_user_query_0,instances_100_1000,0.00,0.112802
676,single_elastic_bm25,101,20825628,mech_label_user_query_1,instances_100_1000,0.00,0.110528
...,...,...,...,...,...,...,...
99523,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,114,23442,mech_label_user_query_1,instances_100000_1000000000,1.00,0.088526
99524,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,114,23442,mech_label_user_query_2,instances_100000_1000000000,1.00,0.079080
99525,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,114,23442,mech_label_user_query_3,instances_100000_1000000000,0.50,0.084768
99526,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,114,23442,mech_label_user_query_4,instances_100000_1000000000,0.25,0.089515


In [14]:
print(df_only_mech_queries[["method_id", "rr", "time"]].groupby(["method_id"], as_index=False).mean().sort_values(by="rr", ascending=False).to_string())


                                                                method_id        rr      time
28  triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.5_x_elastic_bm25_0.1  0.832564  0.188197
23  triple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.6_x_elastic_bm25_0.1  0.831008  0.185811
27  triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.4_x_elastic_bm25_0.2  0.829582  0.188750
22  triple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.5_x_elastic_bm25_0.2  0.827815  0.189693
17  triple_fusion_qdrant_dense_0.2_x_qdrant_sparse_0.7_x_elastic_bm25_0.1  0.826234  0.187460
32  triple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.4_x_elastic_bm25_0.1  0.825458  0.189945
16  triple_fusion_qdrant_dense_0.2_x_qdrant_sparse_0.6_x_elastic_bm25_0.2  0.823393  0.187249
31  triple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.3_x_elastic_bm25_0.2  0.820178  0.191231
46                      tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6  0.819531  0.190523
44                      tuple_fusion_qdrant_dense_0.3_x_qdra

### Only labels

In [15]:
df_only_mech_queries_labels_only = df_only_mech_queries[df_only_mech_queries["query_id"].isin(["mech_label"])]
df_only_mech_queries_labels_only

,method_id,user_id,class_id,query_id,class_category,rr,time
672,single_elastic_bm25,0,20825628,mech_label,instances_100_1000,1.00,0.044602
681,single_elastic_bm25,0,2066754,mech_label,instances_1000_10000,1.00,0.052587
690,single_elastic_bm25,0,34038,mech_label,instances_10000_100000,0.25,0.042479
699,single_elastic_bm25,0,7725634,mech_label,instances_100000_1000000000,1.00,0.078395
708,single_elastic_bm25,0,602884,mech_label,children_10000_100000,1.00,0.072098
...,...,...,...,...,...,...,...
99435,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,0,8016240,mech_label,instances_100_1000,0.50,0.062902
99444,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,0,23442,mech_label,instances_100000_1000000000,1.00,0.052739
99453,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,0,30022,mech_label,instances_1000_10000,1.00,0.070665
99462,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,0,130019,mech_label,instances_10000_100000,1.00,0.063478


In [16]:
print(df_only_mech_queries_labels_only[["method_id", "rr", "time"]].groupby(["method_id"], as_index=False).mean().sort_values(by="rr", ascending=False).to_string())

                                                                method_id        rr      time
23  triple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.6_x_elastic_bm25_0.1  0.933631  0.131741
17  triple_fusion_qdrant_dense_0.2_x_qdrant_sparse_0.7_x_elastic_bm25_0.1  0.933631  0.135877
40                      tuple_fusion_qdrant_dense_0.1_x_qdrant_sparse_0.9  0.932738  0.137034
2                                                    single_qdrant_sparse  0.932143  0.066319
65                      tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1  0.923214  0.068255
10  triple_fusion_qdrant_dense_0.1_x_qdrant_sparse_0.8_x_elastic_bm25_0.1  0.923214  0.134718
42                      tuple_fusion_qdrant_dense_0.2_x_qdrant_sparse_0.8  0.920833  0.135943
44                      tuple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.7  0.918750  0.135366
46                      tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6  0.917758  0.140172
28  triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.5_x_ela

### Only descriptions

In [17]:
df_only_mech_queries_desc_only = df_only_mech_queries[df_only_mech_queries["query_id"].isin(["mech_desc"])]
df_only_mech_queries_desc_only

,method_id,user_id,class_id,query_id,class_category,rr,time
673,single_elastic_bm25,0,20825628,mech_desc,instances_100_1000,0.0,0.126199
682,single_elastic_bm25,0,2066754,mech_desc,instances_1000_10000,0.2,0.086679
691,single_elastic_bm25,0,34038,mech_desc,instances_10000_100000,0.0,0.078374
700,single_elastic_bm25,0,7725634,mech_desc,instances_100000_1000000000,0.2,0.058000
709,single_elastic_bm25,0,602884,mech_desc,children_10000_100000,0.0,0.091256
...,...,...,...,...,...,...,...
99436,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,0,8016240,mech_desc,instances_100_1000,0.5,0.124134
99445,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,0,23442,mech_desc,instances_100000_1000000000,1.0,0.130381
99454,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,0,30022,mech_desc,instances_1000_10000,1.0,0.082419
99463,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,0,130019,mech_desc,instances_10000_100000,1.0,0.119998


In [18]:
print(df_only_mech_queries_desc_only[["method_id", "rr", "time"]].groupby(["method_id"], as_index=False).mean().sort_values(by="rr", ascending=False).to_string())

                                                                method_id        rr      time
40                      tuple_fusion_qdrant_dense_0.1_x_qdrant_sparse_0.9  0.873457  0.190836
2                                                    single_qdrant_sparse  0.872671  0.089913
10  triple_fusion_qdrant_dense_0.1_x_qdrant_sparse_0.8_x_elastic_bm25_0.1  0.872531  0.189748
65                      tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1  0.870202  0.113551
42                      tuple_fusion_qdrant_dense_0.2_x_qdrant_sparse_0.8  0.869585  0.188336
17  triple_fusion_qdrant_dense_0.2_x_qdrant_sparse_0.7_x_elastic_bm25_0.1  0.869444  0.188300
44                      tuple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.7  0.867424  0.189607
46                      tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6  0.866512  0.191802
23  triple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.6_x_elastic_bm25_0.1  0.864969  0.185625
28  triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.5_x_ela

### Only labels with descriptions

In [19]:
df_only_mech_queries_label_desc_only = df_only_mech_queries[df_only_mech_queries["query_id"].isin(["mech_label_desc"])]
df_only_mech_queries_label_desc_only

,method_id,user_id,class_id,query_id,class_category,rr,time
674,single_elastic_bm25,0,20825628,mech_label_desc,instances_100_1000,0.500000,0.139791
683,single_elastic_bm25,0,2066754,mech_label_desc,instances_1000_10000,0.200000,0.122716
692,single_elastic_bm25,0,34038,mech_label_desc,instances_10000_100000,0.066667,0.105941
701,single_elastic_bm25,0,7725634,mech_label_desc,instances_100000_1000000000,1.000000,0.093617
710,single_elastic_bm25,0,602884,mech_label_desc,children_10000_100000,0.090909,0.139669
...,...,...,...,...,...,...,...
99437,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,0,8016240,mech_label_desc,instances_100_1000,1.000000,0.137422
99446,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,0,23442,mech_label_desc,instances_100000_1000000000,1.000000,0.145685
99455,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,0,30022,mech_label_desc,instances_1000_10000,1.000000,0.091105
99464,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,0,130019,mech_label_desc,instances_10000_100000,1.000000,0.160666


In [20]:
print(df_only_mech_queries_label_desc_only[["method_id", "rr", "time"]].groupby(["method_id"], as_index=False).mean().sort_values(by="rr", ascending=False).to_string())

                                                                method_id        rr      time
65                      tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1  1.000000  0.138232
50                      tuple_fusion_qdrant_dense_0.6_x_qdrant_sparse_0.4  1.000000  0.198456
32  triple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.4_x_elastic_bm25_0.1  1.000000  0.201137
37  triple_fusion_qdrant_dense_0.7_x_qdrant_sparse_0.2_x_elastic_bm25_0.1  1.000000  0.197762
28  triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.5_x_elastic_bm25_0.1  1.000000  0.199770
38  triple_fusion_qdrant_dense_0.8_x_qdrant_sparse_0.1_x_elastic_bm25_0.1  1.000000  0.198438
40                      tuple_fusion_qdrant_dense_0.1_x_qdrant_sparse_0.9  1.000000  0.200857
42                      tuple_fusion_qdrant_dense_0.2_x_qdrant_sparse_0.8  1.000000  0.198100
23  triple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.6_x_elastic_bm25_0.1  1.000000  0.193896
44                      tuple_fusion_qdrant_dense_0.3_x_qdra

### Only labels with user queries

In [21]:
df_only_mech_queries_label_user_only = df_only_mech_queries[~df_only_mech_queries["query_id"].isin(["mech_desc", "mech_label", "mech_label_desc"])]
df_only_mech_queries_label_user_only

,method_id,user_id,class_id,query_id,class_category,rr,time
675,single_elastic_bm25,101,20825628,mech_label_user_query_0,instances_100_1000,0.00,0.112802
676,single_elastic_bm25,101,20825628,mech_label_user_query_1,instances_100_1000,0.00,0.110528
677,single_elastic_bm25,101,20825628,mech_label_user_query_2,instances_100_1000,0.00,0.072722
678,single_elastic_bm25,101,20825628,mech_label_user_query_3,instances_100_1000,0.00,0.073281
679,single_elastic_bm25,101,20825628,mech_label_user_query_4,instances_100_1000,0.00,0.078270
...,...,...,...,...,...,...,...
99523,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,114,23442,mech_label_user_query_1,instances_100000_1000000000,1.00,0.088526
99524,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,114,23442,mech_label_user_query_2,instances_100000_1000000000,1.00,0.079080
99525,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,114,23442,mech_label_user_query_3,instances_100000_1000000000,0.50,0.084768
99526,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,114,23442,mech_label_user_query_4,instances_100000_1000000000,0.25,0.089515


In [22]:
print(df_only_mech_queries_label_user_only[["method_id", "rr", "time"]].groupby(["method_id"], as_index=False).mean().sort_values(by="rr", ascending=False).to_string())

                                                                method_id        rr      time
27  triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.4_x_elastic_bm25_0.2  0.813320  0.191920
28  triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.5_x_elastic_bm25_0.1  0.809826  0.191730
22  triple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.5_x_elastic_bm25_0.2  0.808211  0.193017
31  triple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.3_x_elastic_bm25_0.2  0.806615  0.194511
23  triple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.6_x_elastic_bm25_0.1  0.806148  0.189682
32  triple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.4_x_elastic_bm25_0.1  0.805120  0.193427
16  triple_fusion_qdrant_dense_0.2_x_qdrant_sparse_0.6_x_elastic_bm25_0.2  0.801597  0.190820
17  triple_fusion_qdrant_dense_0.2_x_qdrant_sparse_0.7_x_elastic_bm25_0.1  0.799849  0.190784
26  triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.3_x_elastic_bm25_0.3  0.796398  0.194415
48                      tuple_fusion_qdrant_dense_0.5_x_qdra

### Overall per category

In [23]:
df_only_mech_queries_per_category_thin = df_only_mech_queries[["method_id", "class_category", "rr", "time"]]
df_only_mech_queries_per_category_thin

,method_id,class_category,rr,time
672,single_elastic_bm25,instances_100_1000,1.00,0.044602
673,single_elastic_bm25,instances_100_1000,0.00,0.126199
674,single_elastic_bm25,instances_100_1000,0.50,0.139791
675,single_elastic_bm25,instances_100_1000,0.00,0.112802
676,single_elastic_bm25,instances_100_1000,0.00,0.110528
...,...,...,...,...
99523,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,instances_100000_1000000000,1.00,0.088526
99524,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,instances_100000_1000000000,1.00,0.079080
99525,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,instances_100000_1000000000,0.50,0.084768
99526,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,instances_100000_1000000000,0.25,0.089515


In [24]:
df_only_mech_queries_overall_per_category = df_only_mech_queries_per_category_thin.groupby(["method_id", "class_category"], as_index=False).mean().sort_values(by=["class_category", "rr"], ascending=False)
print(df_only_mech_queries_overall_per_category.to_string())

                                                                 method_id               class_category        rr      time
143  triple_fusion_qdrant_dense_0.2_x_qdrant_sparse_0.7_x_elastic_bm25_0.1           instances_100_1000  0.821640  0.192958
191  triple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.6_x_elastic_bm25_0.1           instances_100_1000  0.816461  0.190894
87   triple_fusion_qdrant_dense_0.1_x_qdrant_sparse_0.8_x_elastic_bm25_0.1           instances_100_1000  0.812375  0.194824
79   triple_fusion_qdrant_dense_0.1_x_qdrant_sparse_0.7_x_elastic_bm25_0.2           instances_100_1000  0.811861  0.200376
183  triple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.5_x_elastic_bm25_0.2           instances_100_1000  0.809848  0.196411
135  triple_fusion_qdrant_dense_0.2_x_qdrant_sparse_0.6_x_elastic_bm25_0.2           instances_100_1000  0.809389  0.192292
231  triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.5_x_elastic_bm25_0.1           instances_100_1000  0.809127  0.193531
223  tri

# Overall - only user queries

In [25]:
df_only_user_queries = df[df["query_id"].str.startswith("user")]
df_only_user_queries

,method_id,user_id,class_id,query_id,class_category,rr,time
0,single_elastic_bm25,1,20825628,user_query_0,instances_100_1000,0.000000,0.115999
1,single_elastic_bm25,1,20825628,user_query_1,instances_100_1000,0.000000,0.069224
2,single_elastic_bm25,1,20825628,user_query_2,instances_100_1000,0.000000,0.050538
3,single_elastic_bm25,1,20825628,user_query_3,instances_100_1000,0.000000,0.081411
4,single_elastic_bm25,1,20825628,user_query_4,instances_100_1000,0.000000,0.255341
...,...,...,...,...,...,...,...
98687,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,14,23442,user_query_1,instances_100000_1000000000,0.111111,0.080108
98688,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,14,23442,user_query_2,instances_100000_1000000000,0.200000,0.066064
98689,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,14,23442,user_query_3,instances_100000_1000000000,0.000000,0.083738
98690,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,14,23442,user_query_4,instances_100000_1000000000,0.000000,0.085917


In [26]:
print(df_only_user_queries[["method_id", "rr", "time"]].groupby(["method_id"], as_index=False).mean().sort_values(by="rr", ascending=False).to_string())

                                                                method_id        rr      time
46                      tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6  0.231868  0.168158
28  triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.5_x_elastic_bm25_0.1  0.230941  0.168258
27  triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.4_x_elastic_bm25_0.2  0.227496  0.169233
22  triple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.5_x_elastic_bm25_0.2  0.227026  0.168511
32  triple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.4_x_elastic_bm25_0.1  0.227021  0.168088
48                      tuple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.5  0.226363  0.164035
50                      tuple_fusion_qdrant_dense_0.6_x_qdrant_sparse_0.4  0.225552  0.166644
23  triple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.6_x_elastic_bm25_0.1  0.225032  0.167513
44                      tuple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.7  0.224636  0.165357
35  triple_fusion_qdrant_dense_0.6_x_qdrant_sparse_0.3_x_ela

### Overall per category

In [27]:
df_only_user_queries_per_category_thin = df_only_user_queries[["method_id", "class_category", "rr", "time"]]
df_only_user_queries_per_category_thin

,method_id,class_category,rr,time
0,single_elastic_bm25,instances_100_1000,0.000000,0.115999
1,single_elastic_bm25,instances_100_1000,0.000000,0.069224
2,single_elastic_bm25,instances_100_1000,0.000000,0.050538
3,single_elastic_bm25,instances_100_1000,0.000000,0.081411
4,single_elastic_bm25,instances_100_1000,0.000000,0.255341
...,...,...,...,...
98687,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,instances_100000_1000000000,0.111111,0.080108
98688,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,instances_100000_1000000000,0.200000,0.066064
98689,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,instances_100000_1000000000,0.000000,0.083738
98690,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,instances_100000_1000000000,0.000000,0.085917


In [28]:
df_only_user_queries_overall_per_category = df_only_user_queries_per_category_thin.groupby(["method_id", "class_category"], as_index=False).mean().sort_values(by=["class_category", "rr"], ascending=False)
print(df_only_user_queries_overall_per_category.to_string())

                                                                 method_id               class_category        rr      time
191  triple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.6_x_elastic_bm25_0.1           instances_100_1000  0.266637  0.168255
231  triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.5_x_elastic_bm25_0.1           instances_100_1000  0.265058  0.168628
391                      tuple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.5           instances_100_1000  0.264845  0.163902
375                      tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6           instances_100_1000  0.263425  0.167922
263  triple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.4_x_elastic_bm25_0.1           instances_100_1000  0.263178  0.168558
183  triple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.5_x_elastic_bm25_0.2           instances_100_1000  0.262788  0.168537
287  triple_fusion_qdrant_dense_0.6_x_qdrant_sparse_0.3_x_elastic_bm25_0.1           instances_100_1000  0.262636  0.169576
359     

### Choosing the best match per class of a user

- The idea is that if user writes the 6 queries, and at least one query find the result, the user can use it to model the data structure.
- We want to choose the best query for each of the user assigned classes.
- We can do it by grouping by: method, user, and class.
  - Then we select a maximum for the groups from reciprocal rank.
  - Then the results will be used to again group by a method. 

In [29]:
df_only_user_queries_max_per_user_class = df_only_user_queries[["method_id", "user_id", "class_id", "rr"]].groupby(["method_id", "user_id", "class_id"], as_index=False).max()
df_only_user_queries_max_per_user_class

,method_id,user_id,class_id,rr
0,single_elastic_bm25,1,34038,0.000000
1,single_elastic_bm25,1,602884,0.000000
2,single_elastic_bm25,1,2066754,0.000000
3,single_elastic_bm25,1,4897819,0.000000
4,single_elastic_bm25,1,7725634,0.000000
...,...,...,...,...
7387,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,14,8016240,0.000000
7388,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,14,11483816,1.000000
7389,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,14,18593264,1.000000
7390,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1,14,28942331,0.333333


### Average rr with only best ranks per classes of users


In [30]:
print(df_only_user_queries_max_per_user_class[["method_id", "rr"]].groupby(["method_id"], as_index=False).mean().sort_values(by="rr", ascending=False).to_string())

                                                                method_id        rr
46                      tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6  0.582642
44                      tuple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.7  0.566155
42                      tuple_fusion_qdrant_dense_0.2_x_qdrant_sparse_0.8  0.560553
28  triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.5_x_elastic_bm25_0.1  0.555654
50                      tuple_fusion_qdrant_dense_0.6_x_qdrant_sparse_0.4  0.555480
23  triple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.6_x_elastic_bm25_0.1  0.553766
48                      tuple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.5  0.552827
17  triple_fusion_qdrant_dense_0.2_x_qdrant_sparse_0.7_x_elastic_bm25_0.1  0.548075
32  triple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.4_x_elastic_bm25_0.1  0.546496
27  triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.4_x_elastic_bm25_0.2  0.545263
40                      tuple_fusion_qdrant_dense_0.1_x_qdrant_sparse_0.9  0

### Number of classes found per method with only best ranks

- In total we have 56 classes and each class is given to two users.
- Each method have a change of scoring 112 found classes.

In [31]:
df_only_user_queries_max_per_user_class_thin = df_only_user_queries_max_per_user_class[["method_id", "rr"]].copy()
# Round up rr meaning the method found the class.
df_only_user_queries_max_per_user_class_thin["rr"] = df_only_user_queries_max_per_user_class_thin["rr"].apply(lambda x: math.ceil(x))
# Group the results by method -> for a method, each class is presented twice (since two users provided the queries).
print(df_only_user_queries_max_per_user_class_thin.groupby(["method_id"], as_index=False).sum().sort_values(by="rr", ascending=False).to_string())


                                                                method_id  rr
50                      tuple_fusion_qdrant_dense_0.6_x_qdrant_sparse_0.4  98
32  triple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.4_x_elastic_bm25_0.1  98
28  triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.5_x_elastic_bm25_0.1  98
27  triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.4_x_elastic_bm25_0.2  98
48                      tuple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.5  98
35  triple_fusion_qdrant_dense_0.6_x_qdrant_sparse_0.3_x_elastic_bm25_0.1  97
31  triple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.3_x_elastic_bm25_0.2  97
26  triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.3_x_elastic_bm25_0.3  97
16  triple_fusion_qdrant_dense_0.2_x_qdrant_sparse_0.6_x_elastic_bm25_0.2  96
15  triple_fusion_qdrant_dense_0.2_x_qdrant_sparse_0.5_x_elastic_bm25_0.3  96
46                      tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6  96
44                      tuple_fusion_qdrant_dense_0.3_x_qdrant_s

### Classes the best methods did not find

- triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.5_x_elastic_bm25_0.1
- tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6

In [32]:
# Triple fusion triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.5_x_elastic_bm25_0.1

df1_triple_best = df_only_user_queries_max_per_user_class[df_only_user_queries_max_per_user_class["method_id"].str.contains("triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.5_x_elastic_bm25_0.1")][["method_id", "class_id", "rr"]].copy()
df1_triple_best["rr"] = df_only_user_queries_max_per_user_class_thin["rr"].apply(lambda x: math.ceil(x))
df1_triple_best[df1_triple_best["rr"] == 0]

,method_id,class_id,rr
3139,triple_fusion_qdrant_dense_0.4_x_qdrant_sparse...,4897819,0
3140,triple_fusion_qdrant_dense_0.4_x_qdrant_sparse...,7725634,0
3141,triple_fusion_qdrant_dense_0.4_x_qdrant_sparse...,18557321,0
3144,triple_fusion_qdrant_dense_0.4_x_qdrant_sparse...,133036,0
3145,triple_fusion_qdrant_dense_0.4_x_qdrant_sparse...,134556,0
3151,triple_fusion_qdrant_dense_0.4_x_qdrant_sparse...,123691918,0
3155,triple_fusion_qdrant_dense_0.4_x_qdrant_sparse...,4897819,0
3160,triple_fusion_qdrant_dense_0.4_x_qdrant_sparse...,133036,0
3165,triple_fusion_qdrant_dense_0.4_x_qdrant_sparse...,20755546,0
3167,triple_fusion_qdrant_dense_0.4_x_qdrant_sparse...,123691918,0


- The class it did not find at all:
    - 4897819 - role
    - 133036 - litography
    - 123691918 - tool

In [33]:
# Tuple fusion tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6

df1_triple_best = df_only_user_queries_max_per_user_class[df_only_user_queries_max_per_user_class["method_id"].str.contains("tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6")][["method_id", "class_id", "rr"]].copy()
df1_triple_best["rr"] = df_only_user_queries_max_per_user_class_thin["rr"].apply(lambda x: math.ceil(x))
df1_triple_best[df1_triple_best["rr"] == 0]

,method_id,class_id,rr
5153,tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6,602884,0
5155,tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6,4897819,0
5156,tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6,7725634,0
5157,tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6,18557321,0
5160,tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6,133036,0
5161,tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6,134556,0
5167,tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6,123691918,0
5171,tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6,4897819,0
5176,tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6,133036,0
5181,tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6,20755546,0


- The classes it did not find at all
    - 4897819 - role
    - 133036 - litography
    - 123691918 - tool

Interestingly, they are the same as before. Looking into the queries, we tried them manually, and it did not find them.
Thinking, maybe it was since the things were too abstract, the users have a big problem with generating the queries to.

# The least found classes

In [34]:
df[["class_id", "class_category", "rr"]].groupby(["class_id", "class_category"], as_index=False).mean().sort_values(by="rr", ascending=False)

,class_id,class_category,rr
12,130019,instances_10000_100000,0.918775
36,11483816,instances_1000_10000,0.787376
28,2819161,instances_100_1000,0.758244
51,51591359,instances_10000_100000,0.755959
41,18593264,instances_100000_1000000000,0.738710
46,23002039,instances_1000_10000,0.728493
43,20755546,ancestors_180_1000000000,0.706572
49,30325611,ancestors_150_180,0.696442
24,1371150,instances_100_1000,0.693112
50,35922124,instances_100_1000,0.692378
